In [1]:
import pandas as pd 
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
import numpy as np
import pickle
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from datetime import date
import csv
import os

# Chargement des données

In [2]:
digits = pd.read_csv("../data-csv/train.csv")
digits.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
y = digits["label"]
x = digits[digits.columns.drop("label")]

# Découpage du jeu de données (futur modèle)

In [4]:
# reparti le set entre un set de train et un set de test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, stratify=y, random_state=42)

In [5]:
# nombre de fois où un label est utilisé dans l'entrainement
y_train.value_counts()

1    3138
7    2950
3    2917
9    2806
2    2799
6    2772
0    2770
4    2729
8    2722
5    2543
Name: label, dtype: int64

In [6]:
# nombre de fois où un label est utilisé dans l'entrainement
y_test.value_counts()

1    1546
7    1453
3    1436
9    1382
2    1378
6    1366
0    1364
4    1344
8    1341
5    1253
Name: label, dtype: int64

In [7]:
# on remarque que y_train.value_counts() + y_test.value_counts() = digits['label'].value_counts()
# la division du set a donc été effectuée correctement 
y_train.value_counts() + y_test.value_counts()

1    4684
7    4403
3    4353
9    4188
2    4177
6    4138
0    4134
4    4073
8    4063
5    3796
Name: label, dtype: int64

# Benchmark 

In [8]:
# entrainement du dummy_classifier
dummy_classifier = DummyClassifier(strategy="uniform")
dummy_classifier.fit(x_train, y_train)

# prediction du dummy classifier
dummy_prediction = dummy_classifier.predict(x_test)
dummy_prediction

# calcule du taux d'erreur du dummy_classifier
conf_mat = confusion_matrix(y_test, dummy_prediction)
acc = conf_mat.diagonal().sum()/conf_mat.sum()
err = 1 - acc
print("taux d'erreur du dummy:", err * 100)

taux d'erreur du dummy: 89.91341991341992


# Entrainement du modèle et prediction

### Decision Tree Classifier

In [9]:
depth = range(2,20)
errors = []

for d in depth:
    modelDecisionTreeClassifier = tree.DecisionTreeClassifier(max_depth=d)
    modelDecisionTreeClassifier.fit(x_train, y_train)

    err = round((1.0 - modelDecisionTreeClassifier.score(x_test, y_test)) * 100, 3)
    errors.append(err)
    

KeyboardInterrupt: 

In [ ]:
plt.plot(errors, 'b')
plt.xlabel('depth')
plt.show()

In [ ]:
OPTIMUM_DEPTH = 12
# entrainement du modèle avec la profondeur optimal
modelDecisionTreeClassifier = tree.DecisionTreeClassifier(max_depth=OPTIMUM_DEPTH)
modelDecisionTreeClassifier.fit(x_train, y_train)

# prediction sur x_test
y_prediction = modelDecisionTreeClassifier.predict(x_test)

In [ ]:
# Taux d'erreur du decisionTreeClassifier
conf_mat = confusion_matrix(y_test, y_prediction)
acc = conf_mat.diagonal().sum()/conf_mat.sum()
err = 1 - acc
print("taux d'erreur:", err * 100)

### Random Forrest Classifier

In [ ]:
depth = range(2,12)
estimators_count = range(2,100)
errors = []
estimators = []
depths = []
err_default = 15

for e in estimators_count:
    estimators.append(e)
    model = RandomForestClassifier(n_estimators=e, n_jobs=-1)
    model.fit(x_train, y_train)

    err = round((1.0 - model.score(x_test, y_test)) * 100, 3)
    errors.append(err)

plt.plot(estimators, errors, 'r')
plt.xlabel('# of trees')
plt.show()

In [ ]:
# Taux d'erreur par rapport au nombre d'arbres
d = {'trees': estimators, 'errors': errors}
pd.DataFrame(data=d).sort_values(by=['errors'])

# Entrainement et prédiction optimals 

In [3]:
# entrainement optimal du modèle 
OPTIMUM_DEPTH = 12
OPTIMUM_ESTIMATORS = 73

model = RandomForestClassifier(n_estimators=OPTIMUM_ESTIMATORS)
model.fit(x_train, y_train)

# prediction sur x_test
y_prediction = model.predict(x_test)

# affichage prédiction
y_prediction

# set d'origine
y_test

NameError: name 'x_train' is not defined

### Calcule du taux d'erreur du RandomForestClassifier

In [23]:
conf_mat = confusion_matrix(y_test, y_prediction)
acc = conf_mat.diagonal().sum()/conf_mat.sum()
err = (1 - acc) * 100
print("taux d'erreur:", err)

taux d'erreur: 3.953823953823954


# Sauvegarde du progrés

In [10]:
progress_dict = digits['label'].value_counts().to_dict()

arr = list()
for key,value in progress_dict.items():
    arr.insert(key, value)

arr.append(err)
arr.append(date.today().strftime('%Y-%m-%d'))

with open('../progress.csv','a') as file:
        if os.stat("../progress.csv").st_size == 0:
            file.write("0,1,2,3,4,5,6,7,8,9,error_percentage,execution_date\n")
        file.write((','.join([str(i) for i in arr]) + '\n'))

NameError: name 'err' is not defined

# Sauvegarde du modèle

In [9]:
filename = "../model.pkl"

pickle.dump(model, open(filename, 'wb'))